In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=8774db241a430f6c3288cf9e9f1060d0a241810f3b76f766dfead6a2b7e893c8
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 27.02 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_202000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_20_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=1012529, Tue Apr 14 11:43:00 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 7.927945247490739e+05 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 7.972091543611221e+05 1.0e+00 2.34e+00  2e+00  2e+00 0:00.1
    3     51 7.052253367024611e+05 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [792794.52474907] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [876480.47772025] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   87   1479 -7.414693702813318e+05 1.3e+00 8.61e-01  8e-01  9e-01 0:03.1
  100   1700 -7.849244757535589e+05 1.3e+00 6.78e-01  7e-01  7e-01 0:03.6
  200   3400 -1.095654936510192e+06 1.3e+00 1.11e-01  1e-01  1e-01 0:07.1
  300   5100 -1.104930676263789e+06 1.3e+00 1.86e-02  2e-02  2e-02 0:10.7
  400   6800 -1.105190280575513e+06 1.3e+00 2.87e-03  3e-03  3e-03 0:14.2
  500   8500 -1.105196010428346e+06 1.3e+00 5.40e-04  5e-04  5e-04 0:17.7
  600  10200 -1.105196183595620e+06 1.4e+00 7.99e-05  7e-05  8e-05 0:21.3
  700  11900 -1.105196187865746e+06 1.4e+00 1.39e-05  1e-05  1e-05 0:24.8
  800  13600 -1.105196187986270e+06 1.4e+00 2.41e-06  2e-06  2e-06 0:28.3
  900  15300 -1.105196187989607e+06 1.4e+00 3.92e-07  4e-07  4e-07 0:31.8
 1000  17000 -1.105196187989689e+06 1.4e+00 6.81e-08  6e-08  6e-08 0:35.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.105196187989690e+06 1.4e+00 3.13e-08  3e-08  3e-08 0:38.9
 1200  20400 -1.105196187989691e+06 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [980953.14023324] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)


    2     34 5.204154452502115e+05 1.0e+00 2.32e+00  2e+00  2e+00 0:00.1
    3     51 5.072999445656632e+05 1.0e+00 2.26e+00  2e+00  2e+00 0:00.1
   77   1309 1.026732222375377e+05 1.3e+00 1.51e+00  1e+00  2e+00 0:03.1
  100   1700 -9.528894637750636e+03 1.3e+00 1.15e+00  1e+00  1e+00 0:04.1
  200   3400 -1.961573445117961e+05 1.4e+00 3.37e-01  3e-01  3e-01 0:08.2
  300   5100 -2.197699154274647e+05 1.4e+00 1.17e-01  1e-01  1e-01 0:12.3
  400   6800 -2.246404851940426e+05 1.9e+00 7.00e-02  6e-02  8e-02 0:16.4
  500   8500 -2.271149957077983e+05 2.6e+00 5.50e-02  5e-02  8e-02 0:20.5
  600  10200 -2.310901334008212e+05 3.6e+00 7.66e-02  7e-02  1e-01 0:24.6
  700  11900 -2.353409255690050e+05 4.2e+00 7.12e-02  6e-02  1e-01 0:28.7
  800  13600 -2.370214349015601e+05 4.2e+00 3.01e-02  3e-02  6e-02 0:32.8
  900  15300 -2.372187683975757e+05 4.2e+00 1.02e-02  9e-03  2e-02 0:36.9
 1000  17000 -2.372376092322752e+05 4.2e+00 3.48e-03  3e-03  6e-03 0:41.0
Iterat #Fevals   function value  axis rat

...

   84   1428 -5.167565398736181e+05 1.3e+00 1.07e+00  1e+00  1e+00 0:03.1
  100   1700 -7.193248696879778e+05 1.3e+00 8.09e-01  8e-01  8e-01 0:03.7
  200   3400 -1.090831524417134e+06 1.3e+00 1.49e-01  1e-01  1e-01 0:07.4
  300   5100 -1.104885279322977e+06 1.3e+00 2.28e-02  2e-02  2e-02 0:11.0
  400   6800 -1.105187471242534e+06 1.3e+00 3.53e-03  3e-03  3e-03 0:14.7
  500   8500 -1.105195990726384e+06 1.4e+00 5.50e-04  5e-04  5e-04 0:18.4
  600  10200 -1.105196183492273e+06 1.3e+00 8.98e-05  8e-05  9e-05 0:22.0
  700  11900 -1.105196187874853e+06 1.4e+00 1.51e-05  1e-05  1e-05 0:25.7
  800  13600 -1.105196187986135e+06 1.4e+00 2.30e-06  2e-06  2e-06 0:29.3
  900  15300 -1.105196187989586e+06 1.4e+00 4.14e-07  4e-07  4e-07 0:33.0
 1000  17000 -1.105196187989689e+06 1.4e+00 6.21e-08  6e-08  6e-08 0:36.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.105196187989691e+06 1.4e+00 2.36e-08  2e-08  2e-08 0:40.3
 1200  20400 -1.105196187989691e+06 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [980953.14023324] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)


    2     34 4.443492988224042e+05 1.0e+00 2.32e+00  2e+00  2e+00 0:00.1
    3     51 4.434188648386784e+05 1.0e+00 2.25e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [523434.53820866] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   76   1292 2.688258669171287e+04 1.2e+00 1.08e+00  1e+00  1e+00 0:03.1
  100   1700 -7.710185993179347e+04 1.2e+00 8.05e-01  8e-01  8e-01 0:04.1
  200   3400 -2.141301261898804e+05 1.3e+00 1.99e-01  2e-01  2e-01 0:08.2
  300   5100 -2.275548928817814e+05 1.7e+00 1.28e-01  1e-01  1e-01 0:12.2
  400   6800 -2.348586042551898e+05 2.1e+00 8.07e-02  7e-02  1e-01 0:16.4
  500   8500 -2.365019068490319e+05 2.2e+00 3.53e-02  3e-02  4e-02 0:20.5
  600  10200 -2.369844138185455e+05 2.4e+00 2.13e-02  2e-02  3e-02 0:24.6
  700  11900 -2.371627350768420e+05 2.6e+00 1.45e-02  1e-02  2e-02 0:28.6
  800  13600 -2.372246718615020e+05 2.9e+00 8.01e-03  7e-03  1e-02 0:32.8
  900  15300 -2.372376840884149e+05 3.0e+00 3.12e-03  3e-03  5e-03 0:36.9
 1000  17000 -2.372399116618267e+05 3.1e+00 1.12e-03  9e-04  2e-03 0:41.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -2.372401647019110e+05 3.2e+00 4.04e-04  3e-04  6e-04 0:45.1
 1200  20400 -2.372401914605304e+05 3.2e+

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F20_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:100])
    SVM_Fun [i] = F20(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F20(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([70811.12286696]), count=array([1])) 0.0020239894482746226
SVM
ModeResult(mode=array([567489.9958033]), count=array([1])) 290744.0044778501
ELN
ModeResult(mode=array([23950.03515327]), count=array([1])) 0.0017221057201811064
